# Plot a candlestick chart from one day of price data

## Imports and set up django environment

In [32]:
from django.utils import timezone
import os
import pandas as pd
import django
from django.db import connection
from IPython.core.display import display, HTML
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, BasicTickFormatter, BoxSelectTool, CustomJS, HoverTool, )
from bokeh.plotting import figure, show
os.chdir('..')

# Allows async calls to django ORM in Jupyter. Required.
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
#django.setup()

from pricedata import models


# Get an hours worth of price data for a symbol

In [55]:
# Get some candle data
to_date = timezone.now()
from_date = to_date - timedelta(minutes=60)
candles = models.Candle.objects.filter(datasource_symbol__datasource__name='MT5', period='1S',
                                           time__gte=from_date,
                                           time__lte=to_date,
                                           datasource_symbol__symbol__name='CADCHF')

candle_data = pd.DataFrame(list(candles.values('datasource_symbol__symbol__name', 'time', 'bid_open',
                                               'bid_high', 'bid_low', 'bid_close', 'ask_open', 'ask_high',
                                               'ask_low', 'ask_close', 'volume')))

candle_data

,datasource_symbol__symbol__name,time,bid_open,bid_high,bid_low,bid_close,ask_open,ask_high,ask_low,ask_close,volume
0,CADCHF,2021-07-23 17:23:38+00:00,0.730870,0.730870,0.730870,0.730870,0.731270,0.731270,0.731270,0.731270,1
1,CADCHF,2021-07-23 17:23:39+00:00,0.730880,0.730880,0.730880,0.730880,0.731270,0.731270,0.731270,0.731270,1
2,CADCHF,2021-07-23 17:23:41+00:00,0.730870,0.730870,0.730870,0.730870,0.731270,0.731270,0.731270,0.731270,1
3,CADCHF,2021-07-23 17:23:42+00:00,0.730880,0.730880,0.730880,0.730880,0.731270,0.731270,0.731270,0.731270,1
4,CADCHF,2021-07-23 17:23:57+00:00,0.730870,0.730870,0.730870,0.730870,0.731260,0.731260,0.731260,0.731260,1
5,CADCHF,2021-07-23 17:23:58+00:00,0.730850,0.730850,0.730850,0.730850,0.731250,0.731250,0.731250,0.731250,1
6,CADCHF,2021-07-23 17:24:01+00:00,0.730800,0.730850,0.730800,0.730850,0.731210,0.731240,0.731210,0.731240,2
7,CADCHF,2021-07-23 17:24:03+00:00,0.730860,0.730860,0.730860,0.730860,0.731240,0.731240,0.731240,0.731240,1
8,CADCHF,2021-07-23 17:24:04+00:00,0.730810,0.730810,0.730810,0.730810,0.731200,0.731200,0.731200,0.731200,1
9,CADCHF,2021-07-23 17:24:09+00:00,0.730830,0.730830,0.730830,0.730830,0.731210,0.731210,0.731210,0.731210,1


## Convert datetime, get symbol and add ohlc columns from bid_

In [56]:
# Get the symbol for the first cell
symbol = candle_data['datasource_symbol__symbol__name'][0]

# Convert time to str
#
candle_data["time"] = pd.to_datetime(candle_data["time"]) #.dt.strftime('%Y-%m-%d %H:%M:%S')

# We will make it easy for ourselves later by adding OHLC columns using the existing bid or ask OHLC columns
# depending on bid_or_ask param
bid_or_ask = 'bid'
candle_data['open'] = candle_data[f'{bid_or_ask}_open']
candle_data['high'] = candle_data[f'{bid_or_ask}_high']
candle_data['low'] = candle_data[f'{bid_or_ask}_low']
candle_data['close'] = candle_data[f'{bid_or_ask}_close']

candle_data

,datasource_symbol__symbol__name,time,bid_open,bid_high,bid_low,bid_close,ask_open,ask_high,ask_low,ask_close,volume,open,high,low,close
0,CADCHF,2021-07-23 17:23:38+00:00,0.730870,0.730870,0.730870,0.730870,0.731270,0.731270,0.731270,0.731270,1,0.730870,0.730870,0.730870,0.730870
1,CADCHF,2021-07-23 17:23:39+00:00,0.730880,0.730880,0.730880,0.730880,0.731270,0.731270,0.731270,0.731270,1,0.730880,0.730880,0.730880,0.730880
2,CADCHF,2021-07-23 17:23:41+00:00,0.730870,0.730870,0.730870,0.730870,0.731270,0.731270,0.731270,0.731270,1,0.730870,0.730870,0.730870,0.730870
3,CADCHF,2021-07-23 17:23:42+00:00,0.730880,0.730880,0.730880,0.730880,0.731270,0.731270,0.731270,0.731270,1,0.730880,0.730880,0.730880,0.730880
4,CADCHF,2021-07-23 17:23:57+00:00,0.730870,0.730870,0.730870,0.730870,0.731260,0.731260,0.731260,0.731260,1,0.730870,0.730870,0.730870,0.730870
5,CADCHF,2021-07-23 17:23:58+00:00,0.730850,0.730850,0.730850,0.730850,0.731250,0.731250,0.731250,0.731250,1,0.730850,0.730850,0.730850,0.730850
6,CADCHF,2021-07-23 17:24:01+00:00,0.730800,0.730850,0.730800,0.730850,0.731210,0.731240,0.731210,0.731240,2,0.730800,0.730850,0.730800,0.730850
7,CADCHF,2021-07-23 17:24:03+00:00,0.730860,0.730860,0.730860,0.730860,0.731240,0.731240,0.731240,0.731240,1,0.730860,0.730860,0.730860,0.730860
8,CADCHF,2021-07-23 17:24:04+00:00,0.730810,0.730810,0.730810,0.730810,0.731200,0.731200,0.731200,0.731200,1,0.730810,0.730810,0.730810,0.730810
9,CADCHF,2021-07-23 17:24:09+00:00,0.730830,0.730830,0.730830,0.730830,0.731210,0.731210,0.731210,0.731210,1,0.730830,0.730830,0.730830,0.730830


## Create candle chart

In [59]:
# Generate candle chart.
source = ColumnDataSource(candle_data)

# Display date and symbol for hover
tooltips = [
    ("Time", "@time"),
    ("Open", "@open"),
    ("High", "@high"),
    ("Low", "@low"),
    ("Close", "@close"),
    ("Volume", "@volume"),
]

p = figure(plot_width=1000, x_axis_type="datetime", toolbar_location='below', tools=[HoverTool()], tooltips=tooltips,
           x_axis_location="above")

inc = candle_data.close > candle_data.open
dec = candle_data.open > candle_data.close

p.segment(candle_data.time, candle_data.high, candle_data.time, candle_data.low, color="black")
# TODO. Width needs to be calculate from period, using MilliSec in period - a bit of spacing
p.vbar(x=candle_data.time[inc], top=candle_data.open[inc], bottom=candle_data.close[inc], width=1000,
       fill_color="#D5E1DD", line_color="black")
p.vbar(x=candle_data.time[dec], top=candle_data.open[dec], bottom=candle_data.close[dec], width=1000,
       fill_color="#F2583E", line_color="black")

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0
p.grid.grid_line_alpha = 0.3

show(p)